In [1]:
import numpy as np

# AES S-Box (Substitution Box)
s_box = [
    [0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76],
    [0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0],
    [0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15],
    [0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75],
    [0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84],
    [0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF],
    [0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8],
    [0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2],
    [0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73],
    [0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB],
    [0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79],
    [0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08],
    [0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A],
    [0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E],
    [0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF],
    [0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16],
]

def sub_bytes(state):
    for i in range(4):
        for j in range(4):
            byte = state[i][j]
            state[i][j] = s_box[(byte >> 4) & 0xF][byte & 0xF]
    return state

def shift_rows(state):
    state[1] = np.roll(state[1], -1)
    state[2] = np.roll(state[2], -2)
    state[3] = np.roll(state[3], -3)
    return state

def mix_single_column(a):
    # Perform the mix column operation on a single column
    temp = a.copy()
    a[0] = (2 * temp[0]) ^ (3 * temp[1]) ^ temp[2] ^ temp[3]
    a[1] = temp[0] ^ (2 * temp[1]) ^ (3 * temp[2]) ^ temp[3]
    a[2] = temp[0] ^ temp[1] ^ (2 * temp[2]) ^ (3 * temp[3])
    a[3] = (3 * temp[0]) ^ temp[1] ^ temp[2] ^ (2 * temp[3])
    return a

def mix_columns(state):
    for i in range(4):
        state[:, i] = mix_single_column(state[:, i])
    return state

def add_round_key(state, round_key):
    for i in range(4):
        for j in range(4):
            state[i][j] ^= round_key[i][j]
    return state

# Initial plaintext and key
plaintext = np.array([
    [0x4C, 0x6F, 0x76, 0x65],  # 'L', 'o', 'v', 'e'
    [0x0C, 0x0C, 0x0C, 0x0C],  # Padding
    [0x0C, 0x0C, 0x0C, 0x0C],  # Padding
    [0x0C, 0x0C, 0x0C, 0x0C]   # Padding
], dtype=np.uint8)

key = np.array([
    [0x2B, 0x7E, 0x15, 0x16],
    [0x28, 0xAE, 0xD2, 0xA6],
    [0xAB, 0xF7, 0x15, 0x88],
    [0x09, 0xCF, 0x4F, 0x3C]
], dtype=np.uint8)

# Initial round key addition
state = add_round_key(plaintext, key)
print("After AddRoundKey:\n", state)

# Apply SubBytes
state = sub_bytes(state)
print("After SubBytes:\n", state)

# Apply ShiftRows
state = shift_rows(state)
print("After ShiftRows:\n", state)

# Apply MixColumns
state = mix_columns(state)
print("After MixColumns:\n", state)

# Final round key addition (for this simplified example)
state = add_round_key(state, key)
print("Final ciphertext:\n", state)


After AddRoundKey:
 [[103  17  99 115]
 [ 36 162 222 170]
 [167 251  25 132]
 [  5 195  67  48]]
After SubBytes:
 [[133 130 251 143]
 [ 54  58  29 172]
 [ 92  15 212  95]
 [107  46  26   4]]
After ShiftRows:
 [[133 130 251 143]
 [ 58  29 172  54]
 [212  95  92  15]
 [  4 107  46  26]]
After MixColumns:
 [[116 103 128 169]
 [137 206 153 212]
 [ 27  96 101 233]
 [105  18  93 160]]
Final ciphertext:
 [[ 95  25 149 191]
 [161  96  75 114]
 [176 151 112  97]
 [ 96 221  18 156]]
